In [1]:
import numpy as np
import numpy.linalg as LA
import random
from collections import defaultdict

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

import keras
%load_ext cython

Using TensorFlow backend.


In [3]:
import numba
import itertools as it
import functools

In [4]:
from min_swaps import min_swaps

In [5]:
@numba.jit
def is_sorted(arr):
    for i in range(len(arr)-1):
        if arr[i] > arr[i+1]:
            return False
    return True

In [6]:
@numba.jit
def swap(arr, i, j):
    temp = arr[i]
    arr[i] = arr[j]
    arr[j] = temp

In [7]:
@functools.lru_cache(maxsize=16)
def swappable_indices(sequence_length):
    return tuple(
        (i,j)
        for i in range(sequence_length-1)
        for j in range(i+1, sequence_length)
    )

In [8]:
@numba.jit
def reachable_sequences(sequence):
    sequence_length = sequence.size
    for i in range(sequence_length-1):
        for j in range(i+1, sequence_length):
            arr = sequence.copy()
            temp = arr[i]
            arr[i] = arr[j]
            arr[j] = temp
            yield arr

In [82]:
%timeit for i in reachable_sequences(a):pass

43.6 µs ± 1.73 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [118]:
from numba import jit, prange, autojit

In [119]:
@jit(nopython=True, parallel=True)
def expectation(arr, maxiter, eps):
    length = arr.size
    old_exp = 0
    exp = 0
    convg = 0
    for n in prange(1, maxiter+1):
        a = arr.copy()
        count = 0
        while not is_sorted(a):
            i = np.random.randint(length)
            j = np.random.randint(length)
            while i==j:
                j = np.random.randint(length)
            temp = a[i]
            a[i] = a[j]
            a[j] = temp
            count += 1
            
        exp = exp + (count - exp)/n
        if abs(exp-old_exp) < eps:
            convg += 1
            if convg > 10: return exp
        else:
            convg = 0
        old_exp = exp
    else:
        return -exp

In [61]:
a = np.array([1,2,3,4,5], dtype='int8')

In [120]:
expectation(a, 1000000, 0.00001)

0.0

In [63]:
b = np.array([2,1,3,5,4], dtype='int8')

In [121]:
expectation(b, 10000000, 0.00001)

132.3265977301951

In [66]:
c = np.array([5,4,3,2,1], dtype='int8')

In [78]:
expectation(c, 10000000, 0.00001)

132.44322789345983

In [69]:
d = np.array([1,4,3,2,5], dtype='int8')

In [76]:
expectation(d, 10000000, 0.00001)

118.99481828524921

In [71]:
e = np.array([2,4,3,5,1], dtype='int8')

In [86]:
expectation(e, 10000000, 0.00001)

133.82620012591573

In [80]:
f = np.array([2,4,1,5,3], dtype='int8')

In [81]:
expectation(f, 10000000, 0.00001)

135.41183044958075

In [77]:
def expct(arr):
    sarr = np.sort(arr)
    diff = np.sum(sarr!=arr)

In [85]:
np.log(132)

4.882801922586371

In [16]:
def z(N):
    env = FixedLengthSequenceEnv(4, (1, 3))
    V = defaultdict(lambda: (0, 1))
    for i in range(N):
        state = env.reset()
        shash = hash(state.tobytes())
        val, n = V[shash]
        while not is_sorted(state):
            action = random.choice(env.actions)
            next_state, reward = env.step(action)

            next_shash = hash(next_state.tobytes())
            next_val, next_n = V[next_shash]
            V[shash] = val + (reward + next_val - val)/n, n+1

            state = next_state
            shash = next_shash
            val, n = next_val, next_n
    
    return V

In [17]:
V = z(10000)

In [18]:
for arr in sorted(set(it.permutations((1,2,3)*4, 4))):
    a = np.array(arr)
    shash = hash(a.tobytes())
    v, n = V[shash]
    if v: print(f'{a}: {v:5.2f}   ({n})')
    

[1 1 2 1]: -3.68   (2450)
[1 2 1 1]: -3.60   (2579)
[1 2 1 2]: -4.45   (3578)
[1 2 2 1]: -4.45   (3485)
[2 1 1 1]: -3.69   (2409)
[2 1 1 2]: -4.36   (3716)
[2 1 2 1]: -4.46   (3556)
[2 1 2 2]: -3.54   (2488)
[2 2 1 1]: -5.27   (4369)
[2 2 1 2]: -3.52   (2567)
[2 2 2 1]: -3.57   (2464)


In [9]:
class FixedLengthSequenceEnv:
    
    def __init__(self, sequence_length, range_limits=None, dtype='int8'):
        self._seq_len = sequence_length
        if range_limits is None:
            dtype_info = np.iinfo(dtype)
            range_limits = (dtype_info.min, dtype_info.max)
        self._range_limits = range_limits
        self._state = np.empty(sequence_length, dtype=dtype)
        
    def reset(self) -> np.ndarray:
        "Generates initial state (a random sequence)"
        self._state = np.random.randint(
            *self._range_limits,
            self._seq_len,
            dtype=self._state.dtype
        )
        return self._state
    
    def is_terminal(self) -> bool:
        return is_sorted(self._state)
    
    def step(self, action: "(index, index)") -> (np.ndarray, int):
        "Performs action, returns next state and reward"
        swap(self._state, *action)
        return self._state, -1
        
    @property
    def n_actions(self) -> int:
        "Return number of unique actions ((i,j)==(j,i))"
        return self._seq_len*(self._seq_len - 1)//2
    
    @property
    def actions(self) -> tuple:
        "List all possible unique actions (swapping indices)"
        if not hasattr(self, '_actions'):
            self._actions = tuple(
                (i,j)
                for i in range(self.sequence_length-1)
                for j in range(i+1, self.sequence_length)
            )
        return self._actions
    
    @property
    def sequence_length(self) -> int:
        return self._seq_len

In [10]:
env = FixedLengthSequenceEnv(5, (0,3))

In [11]:
@numba.jit
def selection_sort_policy(sequence):
    swaps = 0
    n = len(sequence)
    for i in range(n-1):
        if sequence[i] > sequence[i+1]:
            break
    else:
        raise Exception('sequence is already sorted')
    return i, i+sequence[i:].argmin()

state = np.random.randint(-10, 10, 16, dtype='int8')
print(state)
count = 0
while not is_sorted(state):
    swap(state, *selection_sort_policy(state))
    count += 1
print(count)

[ 4  4 -4 -2 -1 -2  2 -2 -6 -8  7 -9  8  8 -3 -2]
18


In [341]:
@numba.jit
def softmax(a):
    exp_a = np.exp(a)
    return exp_a/np.sum(exp_a)

In [343]:
%timeit softmax(a)

851 ns ± 3.75 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [339]:
a

array([-8, -7, -6, -5, -4, -3, -2, -1,  0,  1,  2,  3,  4,  5,  6,  7,  8])

In [67]:
softmax(a/np.linalg.norm(a))

array([0.04007438, 0.04287594, 0.04587337, 0.04908033, 0.0525115 ,
       0.05618253, 0.0601102 , 0.0601102 , 0.0601102 , 0.0601102 ,
       0.0601102 , 0.06880849, 0.06880849, 0.06880849, 0.06880849,
       0.06880849, 0.06880849])

In [68]:
softmax(a/np.sum(np.abs(a)))

array([0.05170695, 0.05292353, 0.05416874, 0.05544324, 0.05674772,
       0.0580829 , 0.0594495 , 0.0594495 , 0.0594495 , 0.0594495 ,
       0.0594495 , 0.0622799 , 0.0622799 , 0.0622799 , 0.0622799 ,
       0.0622799 , 0.0622799 ])

In [69]:
softmax(a/a.std())

array([0.00472901, 0.00700357, 0.01037213, 0.0153609 , 0.02274916,
       0.03369101, 0.04989566, 0.04989566, 0.04989566, 0.04989566,
       0.04989566, 0.10943599, 0.10943599, 0.10943599, 0.10943599,
       0.10943599, 0.10943599])

In [ ]:
def policy_eps_greedy(state_values, state_index, state, eps):
    if np.random.random() < eps:
        max_val = -1e20
        for s, indices in reachable_sequences(state):
            
            
    swappable_indices(state.size):

In [708]:
def policy_sampling(state_values, state_index, state, eps, shift_expectations=False):
    q = np.asarray([
        state_values[arr.tobytes()]
        for arr in reachable_sequences(state)
    ])
    
    if shift_expectations:
        q -= q.min()
        
    if eps == 0:
        action_index = np.argmax(q)
    else:
        norm = np.linalg.norm(q) or 1.0
        # (1-eps)/eps == 0->+inf, 0.5->1 1->0
        scale = (1-eps)/(eps*norm)
        probabilities = softmax(scale*q)
        action_index = np.random.choice(q.size, p=probabilities)
    return swappable_indices(state.size)[action_index]
        

In [722]:
V = defaultdict(lambda: 0)

In [677]:
s=[policy_sampling(z, a.tobytes(), a, 0.) for i in range(10000)]

ss=collections.Counter(s)

sorted(ss.items(), key=lambda i: i[1])

AssertionError: Failed in object mode pipeline (step: object mode frontend)


In [117]:
p=  softmax(a/np.linalg.norm(a))
p

array([0.04007438, 0.04287594, 0.04587337, 0.04908033, 0.0525115 ,
       0.05618253, 0.0601102 , 0.0601102 , 0.0601102 , 0.0601102 ,
       0.0601102 , 0.06880849, 0.06880849, 0.06880849, 0.06880849,
       0.06880849, 0.06880849])

In [125]:
np.random.choice(a.size, p=p)

6

In [693]:
def td0_controll(
    env,
    policy,
    alpha,
    maxiters: 'number of iterations, no early stopping',
#     policy: 'Callable(q, state_index, eps) -> action_index',
    eps_schedule: 'Callable(iteration, previous_eps?) -> eps',
#     alpha_schedule: 'Callable(episode_num, previous_alpha?) -> alpha',
    gamma: 'constant discount factor',
    V = None,
) -> 'action-value table - 2D array: state_indices×action_indices':
    
    if V is None:
        V = defaultdict(lambda: 0.0)
        
    eps = None
#     alpha = None
    
    for i in range(1, maxiters+1):
        eps = eps_schedule(i, eps)
#         alpha = alpha_schedule(i, alpha)

        state = env.reset()
        state_index = state.tobytes()
        action = policy(V, state_index, state, eps)
        while not env.is_terminal():
            
            next_state, reward = env.step(action)
            next_state_index = next_state.tobytes()
            
            next_action = policy(V, next_state_index, next_state, eps)
            
            v = V[state_index]
            nv = V[next_state_index]
            V[state_index] = v + alpha*(reward + gamma*nv - v)
            
            state = next_state
            state_index = next_state_index
            action = next_action

    return V


In [896]:
env = FixedLengthSequenceEnv(8, (0, 3))

In [897]:
V = td0_controll(
    env=env,
    policy=policy_sampling,
    alpha=0.1,
    maxiters=10000,
    eps_schedule=lambda i,e: 0.9995*(e or 1.0),
    gamma=0.9,
#     V=V
)

In [898]:
len(V)


6561

In [899]:
env.reset()

array([2, 2, 0, 1, 1, 0, 2, 1], dtype=int8)

In [953]:
state = np.array([2, 2, 0, 1, 1, 0, 2, 1], dtype='int8')
state_orig = state.copy()
print(state)
steps = 0
while not is_sorted(state):
    action = policy_sampling(V, state.tobytes(), state, 0.01, True)
    swap(state, *action)
    steps += 1
print(steps)

[2 2 0 1 1 0 2 1]
3


In [922]:
V

defaultdict(<function __main__.td0_controll.<locals>.<lambda>()>,
            {b'\x01\x01\x00\x00\x00\x02\x01\x00': -8.491365520189774,
             b'\x00\x01\x01\x00\x00\x02\x01\x00': -8.47097718158421,
             b'\x00\x01\x00\x01\x00\x02\x01\x00': -8.326142664593391,
             b'\x00\x01\x00\x00\x01\x02\x01\x00': -8.141449887767156,
             b'\x02\x01\x00\x00\x00\x01\x01\x00': -7.448301813417347,
             b'\x00\x01\x00\x00\x00\x02\x01\x01': -7.529893328214544,
             b'\x01\x00\x01\x00\x00\x02\x01\x00': -8.444965050097364,
             b'\x01\x00\x00\x01\x00\x02\x01\x00': -8.461848564084276,
             b'\x01\x00\x00\x00\x01\x02\x01\x00': -6.717123525256707,
             b'\x01\x02\x00\x00\x00\x01\x01\x00': -7.441338223570153,
             b'\x01\x00\x00\x00\x00\x02\x01\x01': -8.038986030715332,
             b'\x01\x01\x02\x00\x00\x00\x01\x00': -8.542886354898517,
             b'\x01\x01\x01\x00\x00\x02\x00\x00': -8.63508650059286,
             b'\x01\x01\x0

In [2]:
import numpy as np

state_values = np.zeros(n_states) # initial guess = 0 value
eligibility = np.zeros(n_states)

lamb = 0.95 # the lambda weighting factor
state = env.reset() # start the environment, get the initial state
# Run the algorithm for some episodes
for t in range(n_steps):
    # act according to policy
    action = policy(state)
    new_state, reward, done = env.step(action)
    # Update eligibilities
    eligibility *= lamb * gamma
    eligibility[state] += 1.0

    # get the td-error and update every state's value estimate
    # according to their eligibilities.
    td_error = reward + gamma * state_values[new_state] - state_values[state]
    state_values = state_values + alpha * td_error * eligibility

    if done:
    state = env.reset()
    else:
    state = new_state


<module 'keras' from '/home/martin/.local/lib/python3.7/site-packages/keras/__init__.py'>

## Exercise 1: TD(0) value function estimation

Implement value function estimation for Sutton & Barto example 4.1 with TD(0) algorithm.

In [4]:
def TD0(
    gw: GridWorld,
    maxiters: 'number of iterations, no early stopping',
    alpha_schedule: 'Callable(episode_num, previous_alpha?) -> alpha',
    gamma: 'constant discount factor',
) -> 'state-value matrix':
    
    V = defaultdict(lambda: 0)
    
    alpha = None
    for i in range(maxiters):

        alpha = alpha_schedule(i, alpha)
        state = gw.get_init_state()
        while not gw.is_terminal(state):
            action = random.choice(gw.get_actions(state))
            new_state, reward = gw.take_action(state, action)
            V[state] += alpha*(reward + gamma*V[new_state] - V[state])
            state = new_state

    return V


In [5]:
def constant_alpha_schedule_factory(alpha):
    return lambda episode, prev_alpha: alpha

## Exercise 2: Implement TD(0) control 

Solve Sutton & Barto example 4.1 with TD(0) control (Sarsa) algorithm. Apply the algorithm to the windy world of Sutton & Barto example 6.5. 

In [7]:
def Sarsa(
    gw: GridWorld,
    maxiters: 'number of iterations, no early stopping',
    policy: 'Callable(q, state_index, eps) -> action_index',
    eps_schedule: 'Callable(iteration, previous_eps?) -> eps',
    alpha_schedule: 'Callable(episode_num, previous_alpha?) -> alpha',
    gamma: 'constant discount factor',
) -> 'action-value table - 2D array: state_indices×action_indices':
    
    Q = np.zeros((gw.get_number_of_states(), gw.get_number_of_actions()))
    eps = None
    alpha = None
    
    for i in range(1, maxiters+1):
        eps = eps_schedule(i, eps)
        alpha = alpha_schedule(i, alpha)

        state = gw.get_init_state()
        state_index = gw.get_state_index(state)
        action_index = policy(Q, state_index, eps)
        while not gw.is_terminal(state):
            
            next_state, reward = gw.take_action(state, gw.which_action(action_index))
            next_state_index = gw.get_state_index(next_state)
            
            next_action_index = policy(Q, next_state_index, eps)
            
            q = Q[state_index, action_index]
            qn = Q[next_state_index, next_action_index]
            Q[state_index, action_index] = q + alpha*(reward + gamma*qn - q)
            
            state = next_state
            state_index = next_state_index
            action_index = next_action_index

    return Q


In [8]:
def print_action_value(Q, gw: GridWorld=GW4p1):
    print('Action-value table:')
    print('State ' + ('  {:>4}  '*4).format(*gw._action.ACTION_LABELS))
    for i in range(gw.get_number_of_states()):
        print('{}  {:6.2f}  {:6.2f}  {:6.2f}  {:6.2f}'.format(gw.which_state(i), *Q[i]))
    print()

## Exercise 4**: TD(lambda)

Implement TD(lambda) algorithm and use it for solving example 6.5. Create a table/plot on the effect of lambda in the performance of the algorithm.

*) - not mandatory

In [17]:
def SarsaLambda(
    gw: GridWorld,
    maxiters: 'number of iterations, no early stopping',
    policy: 'Callable(q, state_index, eps) -> action_index',
    eps_schedule: 'Callable(iteration, previous_eps?) -> eps',
    alpha_schedule: 'Callable(episode_num, previous_alpha?) -> alpha',
    gamma: 'constant discount factor',
    lambda_,
    max_episode_length = 200,
) -> 'action-value table - 2D array: state_indices×action_indices':
    
    sa_dims = (gw.get_number_of_states(), gw.get_number_of_actions())
    Q = np.zeros(sa_dims)
    eps = None
    alpha = None
    
    for i in range(1, maxiters+1):
        E = np.zeros(sa_dims)
        
        eps = eps_schedule(i, eps)
        alpha = alpha_schedule(i, alpha)

        state = gw.get_init_state()
        state_index = gw.get_state_index(state)
        action_index = policy(Q, state_index, eps)
        j = 0
        while not gw.is_terminal(state)  and j < max_episode_length:
            j += 1
            
            next_state, reward = gw.take_action(state, gw.which_action(action_index))
            next_state_index = gw.get_state_index(next_state)
            
            next_action_index = policy(Q, next_state_index, eps)
            
            delta = reward + gamma*Q[next_state_index, next_action_index] - Q[state_index, action_index]
            
            E[state_index, action_index] += 1
            
            Q += alpha*delta*E
            E *= gamma*lambda_
            
            state = next_state
            state_index = next_state_index
            action_index = next_action_index
        
    return Q
